In [ ]:
import os, json
from tqdm import tqdm
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain.tools import StructuredTool
from typing import List, Dict, Optional, Any, Literal
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from together import Together
load_dotenv()


In [ ]:
new_collection = {
    "title": "Ramen Noodles Recipe and Steak Cooking Guide",
    "content": """**Title:** Ramen Noodles Recipe and Steak Cooking Guide
**Introduction:**
Ramen noodles are a popular Japanese dish that can be customized to suit various tastes. This recipe provides a basic guide on how to prepare a delicious and flavorful ramen noodle dish. Additionally, we've included a guide on how to cook a perfect steak.

**Ramen Noodles Recipe:**
**Introduction:**
Ramen noodles are a popular Japanese dish that can be customized to suit various tastes. This recipe provides a basic guide on how to prepare a delicious and flavorful ramen noodle dish.

**Ingredients:**
- 1 package of ramen noodles
- 2 cups of water
- 1 tablespoon of vegetable oil
- 1 small onion, diced
- 2 cloves of garlic, minced
- 1 cup of mixed vegetables (e.g., carrots, green onions, mushrooms)
- 1 teaspoon of soy sauce
- 1 teaspoon of sesame oil
- Salt and pepper to taste
- Optional: boiled egg, sliced pork, or green onions for topping

**Instructions:**
1. **Prepare the Broth:** Bring the water to a boil in a large pot.
2. **Cook the Noodles:** Add the ramen noodles and cook according to the package instructions. Typically, this involves boiling for 2-3 minutes or until the noodles are slightly undercooked.
3. **Sauté the Aromatics:** In a separate pan, heat the vegetable oil over medium heat. Add the diced onion and minced garlic and sauté until softened, about 3-4 minutes.
4. **Add Mixed Vegetables:** Add the mixed vegetables to the pan and cook until they're tender, about 4-5 minutes.
5. **Combine and Season:** Combine the cooked noodles, vegetable mixture, soy sauce, and sesame oil in a bowl. Season with salt and pepper to taste.
6. **Serve:** Serve hot, with optional toppings such as boiled egg, sliced pork, or green onions.

**Tips and Variations:**
- For added protein, consider adding cooked chicken, beef, or tofu.
- Experiment with different seasonings, such as chili flakes or grated ginger, to give your ramen a unique flavor.
- Use low-sodium broth or reduce the amount of soy sauce to decrease the dish's salt content.

**Steak Cooking Guide:**
**Introduction:**
Cooking a perfect steak can be a challenge, but with the right techniques and ingredients, you can achieve a delicious and tender steak.

**Ingredients:**
- 1-2 steaks (depending on size and preference)
- 2 tablespoons of olive oil
- 1 teaspoon of salt
- 1/2 teaspoon of black pepper
- 1/2 teaspoon of garlic powder (optional)
- 1/2 teaspoon of paprika (optional)

**Instructions:**
1. **Prepare the Steak:** Bring the steak to room temperature by leaving it out for 30 minutes to 1 hour before cooking.
2. **Season the Steak:** Rub the steak with olive oil, salt, black pepper, garlic powder, and paprika (if using).
3. **Heat the Pan:** Heat a skillet or grill pan over high heat. Add a small amount of oil to the pan and swirl it around.
4. **Sear the Steak:** Add the steak to the pan and sear for 2-3 minutes per side, depending on the thickness of the steak and your desired level of doneness.
5. **Finish Cooking:** After searing the steak, reduce the heat to medium-low and continue cooking to your desired level of doneness.
6. **Let it Rest:** Once the steak is cooked, remove it from the heat and let it rest for 5-10 minutes before slicing and serving.

**Tips and Variations:**
- Use a meat thermometer to ensure the steak is cooked to your desired level of doneness.
- Let the steak rest for a few minutes before slicing to allow the juices to redistribute.
- Experiment with different seasonings and marinades to give your steak a unique flavor."""
}

In [ ]:
together_api_key = os.environ.get("TOGETHER_API_KEY")
groq_api_key = os.environ.get("GROQ_API_KEY")
model = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"
max_tokens = 1024
temperature = 0.6
client = Together(api_key=together_api_key)

In [ ]:
refiner_agent_format = """{
    "agent": "Refiner Agent",
    "content": "content (keep formating)"
}
"""
refiner_agent_format_resp =  """{
    "sender": "Refiner Agent",
    "content": "content (keep formating)"
}"""

database_agent_format_retrieve = """{
    "agent": "Database Agent",
    "task": "retrieve",
    "title": "Title note" or "None"
}"""

database_agent_format_save = """{
    "agent": "Database Agent",
    "task": "save",
    "title": "Title note",
    "content": "Finalized Content (keep formating)"
}"""

database_agent_format_deletion = """{
    "agent": "Database Agent",
    "task": "delete",
    "title": "Title note"
}"""

database_agent_format_input = """
    "agent": "Database Agent",
    "task": ("save" or "retrieve" or "delete"),
    "title": "Title note" or "None"
    "content": "Content note" or "None"
"""
database_agent_format_resp = """
    "task": ("save" or "retrieve" or "delete"),
    "title": "Title note" or "None",
    "content": "Content note" or "None"
"""

In [ ]:
class RefinerSchema(BaseModel):
    refined_content: str
    
class EvaluatorSchema(BaseModel):
    status: Literal["PASS", "NEED REVISION"]
    revision: str

class DatabaseSchema(BaseModel):
    task: str
    title: Optional[str]
    content: Optional[str]
    
manager_prompt = """
You are Naufal's personal assistant and the manager of two agents: the **Refining Agent** and the **Database Agent**. Your responsibilities are:

### Roles:
1. **Personal Assistant**  
   Act as a helpful, engaging companion. Proactively assist Naufal in creating, designing, or taking notes on topics of interest.

2. **Agent Manager**  
   Coordinate two specialized agents:
   - **Refining Agent**: Improves note content for clarity and informativeness.
   - **Database Agent**: Stores finalized notes and retrieves them when needed.

### Workflow:

**1. Initiation**  
Start as Naufal’s assistant. Suggest taking a note if relevant to the conversation.

**2. Note Creation & Management**
- **If Naufal provides content at first**:
  - Recompile it again as a note with contents as same as contents that naufal provided.
  - Ask if he wants it refined (no JSON schema shown when you ask).
    - If **yes**, respond **only** in JSON: `{refiner_agent_format}`
    - If **no**, resume the conversation.
- **If no content is provided**:
  - Suggest relevant topics.
  - Ask if naufal wants to create a note now or later.
    -if **yes**, show the draft to naufal first and inform him he can modify this draft throughout chat.
    -if **no**, continue conversation and recommend him to create a note later on.
  - if Naufal wants to refine the created note. (no JSON schema shown when you ask).
    - If **yes**, respond **only** in JSON: `{refiner_agent_format}`
    - If **no**, continue the conversation.

**3. Note Refinement**
- You’ll receive refined content in format: `{refiner_agent_format_resp}`
- Present it to Naufal and ask for feedback:
  - If **satisfied**, continue the conversation and memorize previous the refined content.
  - If **not satisfied**, respond **only** in JSON: `{refiner_agent_format}`

**4. Previous Created Notes**
- If naufal asks you to show him either the draft or the refined notes, check your previous conversation first then responds to naufal back. Do not call any agents/tools.

**5. Note Saving to Database**
-If naufal does not show an action that he wants to save the note, do not ask him 'want to save the not or not'.
-If you recommend naufal to save the note or not, just ask the question without showing json format for the database agent.
    - If naufal says **yes**, Respond **only** in JSON: `{database_agent_format_save}`. Once you received saving successfull notification from Database Agent, inform the message to naufal.
    - If **no**, continue the conversation and memorize the refined content.
-If Naufal asks to save the created note:
  - Respond **only** in JSON: `{database_agent_format_save}`.
  - Once you received saving successfull notification from Database Agent, inform the message to naufal.

**6. Note Retrieval from Database**
- When Naufal asks to retrieve notes that are saved in database:
  - Respond **only** in JSON: `{database_agent_format_retrieve}`
  - Once a list of saved notes is returned, ask which one he wants.
  - Then respond **only** in JSON: `{database_agent_format_retrieve}` again."""

refiner_prompt = """
You are a competent refiner agent. You work with: your **manager** who is the main & frontline agent for Naufal and your other partner is **Database** agent who will save & retrive notes.
Your only main job/task is to refine the note's content that your manager orders you to do. You will be provided with 3 context: a task, a previous generation, and a revision. 
task is the order that your manager gives, previous generation is your previous refined content, and revision is the feedback from evaluator that you need to fix it. if the & previous generation & revision are none, that indicates it is the first time of the step.
You dont need to think very hard, just do it as best & fast as you can since your response will be evaluated by an evaluator AI later. You dont need to call the evaluator AI yourself, it will be called by the system.
Reflect to the evaluator's revision and improve your answer.

task:

Please refine this content:

{task}

previous generation:

{previous_generation}

revision:

{revision}

Only output JSON with the following schema:

{schema}
"""

evaluator_prompt = """
You are a competent evaluator. Your task is to assess the quality of a `previous_generation` provided in response to a specific `task` by a refiner agent who is naufal personal agent.
Please provide a detailed revision to improve the `previous_generation`. Your evaluation will focus on the following aspects:

**Evaluation Criteria:**
Carefully evaluate the `previous_generation` based on the following:
1.  **Structure & Clarity:** Is it well-organized, easy to understand, and logically structured?
2.  **Adding more nuance** Is it the content added some nuance to make it look more expressive?
2.  **Conciseness:** Is it to the point, without unnecessary verbosity?
3.  **Formatting:** Does the answer use formatting (e.g., bullet points, bolding, lists) effectively and appropriately?
4.  **Word Choice & Tone:** Is the language precise and suitable?

Only output the evaluation criteria with checker for every list above (complete or not) so that the refiner agent will know where to fix it for the revision.
if all the evaluation criteria are marked complete, set status to "PASS" else "NEED REVISION".

previous generation:

{previous_generation}

Decide if the content is "PASS" or "NEED REVISION", Only output JSON with the following schema:

{schema}
"""

database_prompt= """
You are a smart & competence database agent who works with: your **manager** who is the main & frontline agent for Naufal and your other partner is **Refiner** agent who will refine naufal's created note.
Your tasks will be saving & retrieving document from NoSQL (MongoDB) database. Your manager will give the order to you with this format: {manager_format}. 'Agent' means your name, 'task' means your task to save or retrive (task should be only either save or retrive not both),
'title' means title of the note (Optional), 'content' means content of the note (Optional). Pay close attention to the manager's order before answering!

Manager's order:

{task}

Only output in JSON format with following schema:

{db_schema} 
"""

In [ ]:
manager_prompt = """
You are Naufal's personal assistant and the manager of two agents: the **Refining Agent** and the **Database Agent**. Your responsibilities are:

### Roles:
1. **Personal Assistant**  
   Act as a helpful, engaging companion. Proactively assist Naufal in creating, designing, or taking notes on topics of interest.

2. **Agent Manager**  
   Coordinate two specialized agents:
   - **Refining Agent**: Improves note content for clarity and informativeness.
   - **Database Agent**: Stores finalized notes and retrieves them when needed.

### Workflow:

**1. Initiation**  
Start as Naufal’s assistant. Suggest taking a note if relevant to the conversation.

**2. Note Creation & Management**
- **If Naufal provides content at first**:
  - Recompile it again as a note with contents as same as contents that naufal provided.
  - Ask if he wants it refined (no JSON schema shown when you ask).
    - If **yes**, respond **only** in JSON: `{refiner_agent_format}`
    - If **no**, resume the conversation.
- **If no content is provided**:
  - Suggest relevant topics.
  - Ask if naufal wants to create a note now or later.
    -if **yes**, show the draft to naufal first and inform him he can modify this draft throughout chat.
    -if **no**, continue conversation and recommend him to create a note later on.
  - if Naufal wants to refine the created note. (no JSON schema shown when you ask).
    - If **yes**, respond **only** in JSON: `{refiner_agent_format}`
    - If **no**, continue the conversation.

**3. Note Refinement**
- You’ll receive refined content in format: `{refiner_agent_format_resp}`
- Present it to Naufal and ask for feedback:
  - If **satisfied**, continue the conversation and memorize previous the refined content.
  - If **not satisfied**, respond **only** in JSON: `{refiner_agent_format}`

**4. Previous Created Notes**
- If naufal asks you to show him either the draft or the refined notes, check your previous conversation first then responds to naufal back. Do not call any agents/tools.

**5. Note Saving to Database**
-Either if Naufal wants to save the note or you recommend naufal to save the note or not, just answer/ask the question without showing json format for the database agent.
    - If naufal says **yes**, Respond **only** in JSON: `{database_agent_format_save}`. Once you received notification from Database Agent, inform the message to naufal.
    - If **no**, continue the conversation and memorize the refined content.

**6. Note Retrieval from Database**
- When Naufal asks to retrieve notes that are saved in database:
  - Respond **only** in JSON: `{database_agent_format_retrieve}`
  - Once a list of saved notes is returned, ask which one he wants.
  
**7. Note Deletion to Database**  
- Do **not** recommend deletion unless Naufal explicitly asks for it.  
- If Naufal asks to delete a note, first check your chat history (internally) to see if you already have a full list of saved-note titles:  
  - **If you do _not_ have a full list**, respond _only_ with this JSON (no extra text before or after):
    {database_agent_format_retrieve}
  - **If you _do_ have a full list** and Naufal has specified one of those titles, respond _only_ with this JSON (no extra text before or after):
    {database_agent_format_deletion}
- **IMPORTANT:** Under _all_ circumstances in this step, the model’s output **must be exactly** the JSON object and **nothing else**—no apologies, no clarifications, no surrounding prose.
"""

In [ ]:
manager_prompt = """
You are Naufal’s personal assistant and the manager of two agents:
  • Refining Agent — polishes note content  
  • Database Agent — saves and retrieves notes  

ROLES
1. Personal Assistant
   – Engage proactively: suggest taking notes, design or discuss topics.
2. Agent Manager
   – Route tasks to the Refining or Database Agent using the prescribed JSON schemas.

WORKFLOW

1. NOTE CREATION  
   • If Naufal supplies content:
     1. Wrap it in a draft note (no JSON shown).  
     2. Ask “Refine this note?” (no JSON shown).  
        – Yes → output exactly `{refiner_agent_format}`  
        – No  → resume conversation.
   • If no content:
     1. Propose topics.  
     2. Ask Naufal to create a note now or later.  
        – Yes → show draft, allow edits; recommend “Refine?” as above.  
        – No  → continue chat; remind later.

2. NOTE REFINEMENT  
   • After receiving `{refiner_agent_format_resp}`, present refined text and ask satisfied or not.  
     – Yes → confirm and remember it.  
     – No  → output exactly `{refiner_agent_format}` again.

3. NOTE SAVING  
   • When Naufal agrees to save:
     – Output exactly `{database_agent_format_save}`  
     – After Database Agent confirms, notify Naufal.  
   • If he declines, remember content and continue.

4. NOTE RETRIEVAL  
   • On retrieval request → output exactly `{database_agent_format_retrieve}`  
   • When list returns → ask which title to act on.

5. NOTE DELETION  
   • Never suggest deletion. Only if requested:
     1. If no title list exists → output exactly `{database_agent_format_retrieve}`. Emit only the JSON object, no extra text.
     2. If list exists and title provided → output exactly `{database_agent_format_deletion}`.
"""


In [ ]:

def refiner_agent(task: Optional[str], previous_generation: Optional[str], revision: Optional[str], history: List, schema:Optional[Any]) -> str:
    """
    Refine a content based on the task, previous generation, revision
    """
    prompt = refiner_prompt.format(
        task=task,
        previous_generation=previous_generation,
        revision=revision,
        schema=schema.model_json_schema()
    )

    history += [{"role": "system", "content": prompt}]

    response = client.chat.completions.create(model=model, temperature=temperature, max_tokens=max_tokens, messages=history,
                                              response_format={"type": "json_object","schema": schema.model_json_schema(),})

    return response.choices[0].message.content, history

def evaluator_agent(previous_answer: Optional[str], schema: Optional[Any], history: List) -> str:

    

    history += [{"role": "system", "content": evaluator_prompt.format(previous_generation=previous_answer, schema=schema.model_json_schema())}]

    response = client.chat.completions.create(model=model, temperature=temperature, messages=history, max_tokens=max_tokens,
                                              response_format={"type": "json_object","schema": schema.model_json_schema(),})
    return response.choices[0].message.content, history


def database_agent(task: str, schema: Optional[Any]):

    message = [{"role": "system", "content": database_prompt.format(task=task, manager_format=database_agent_format_input, db_schema=database_agent_format_resp)}]

    response = client.chat.completions.create(model=model, temperature=temperature, max_tokens=max_tokens, messages=message,
                                              response_format={"type": "json_object","schema": schema.model_json_schema(),})
    
    return response.choices[0].message.content


In [ ]:
message_history = [{"role": "system", "content": manager_prompt.format(refiner_agent_format=refiner_agent_format, refiner_agent_format_resp=refiner_agent_format_resp,
                                                                       database_agent_format_save=database_agent_format_save, database_agent_format_retrieve=database_agent_format_retrieve,
                                                                       database_agent_format_deletion=database_agent_format_deletion)}]

while True:

    user_input = input_preprocessing(input("Ask anything:"))

    if user_input in ['exit', 'q']:
        break

    print("=======================HUMAN MESSAGE============================")
    print(user_input)
    message_history.append({"role": "user", "content": user_input})

    print("========================AI MESSAGE==============================")

    llm_response_complete = ""

    for chunk in client.chat.completions.create(model=model, max_tokens=max_tokens, temperature=temperature, stream=True, messages=message_history):

        response = chunk.choices[0].delta.content or ""

        print(response, end="", flush=True)
        llm_response_complete += response
    
    message_history.append({"role": "assistant", "content": llm_response_complete})

    print()

    try:
        json_response = json.loads(llm_response_complete, strict=False)
        agent = True

    except:
        agent = False


    if agent:

        agent_name = json_response.get("agent").lower().strip()
        
        if agent_name == "refiner agent":
            json_response = json.loads(llm_response_complete, strict=False)
            refine_agent_history = []
            evaluator_agent_history = []

            print("======================System Message============================")
            print("Calling Refiner Agent...")
            print("======================System Message============================")
            
            first_step, history = refiner_agent(task=json_response.get("content"), history=refine_agent_history, previous_generation=None, revision=None, schema=RefinerSchema)
            prev_step = first_step

            for num_ref in tqdm(range(5), desc="Refining content...", leave=True):
                revision, feedback_history = evaluator_agent(
                    previous_answer=json.loads(prev_step)['refined_content'],
                    schema=EvaluatorSchema,
                    history=evaluator_agent_history
                )

                status = json.loads(revision)["status"]
                feedback = json.loads(revision)["revision"]

                print(status)

                if status == "PASS":
                    break

                prev_step, refined_content_history = refiner_agent(
                    task=json_response.get("content"),
                    previous_generation=json.loads(prev_step)['refined_content'],
                    revision=feedback,
                    history=refine_agent_history,
                    schema=RefinerSchema
                )

                refined_content_history.append({"role": "system", "content":json.loads(prev_step)['refined_content']})

            refiner_agent_resp = str(dict({"sender": "Refiner_Agent", "content": json.loads(prev_step)['refined_content']}))

            message_history.append({"role": "system", "content": refiner_agent_resp})


            print("======================System Message============================")
            print("Sending back to main agent...")
            print("========================AI MESSAGE==============================")

            for chunk in client.chat.completions.create(model=model, max_tokens=max_tokens, temperature=temperature, stream=True, messages=message_history):

                response = chunk.choices[0].delta.content or ""

                print(response, end="", flush=True)
                llm_response_complete += response

            print()

        else:
            print("======================System Message============================")
            print("Calling Database Agent...")
            print("======================System Message============================")
            db_agent_resp = json.loads(database_agent(task=json_response, schema=DatabaseSchema), strict=False)

            print(db_agent_resp, flush=True)

            if db_agent_resp.get('task').lower().strip() == "save":

                title_note = db_agent_resp.get('title').strip()
                content_note = db_agent_resp.get('content').strip()

                assert (title_note != "None") and (content_note != "None"), "Title and content must not be None"

                new_collection = {"title": title_note, "content": content_note}
                status = collections.insert_one(new_collection)

                assert status.__getstate__()[-1]['_WriteResult__acknowledged'], "Fail to write to database"

                feedback_db = {"role": "system", "content": str({"sender": "Database Agent", "saving_status": "The note has been successfully saved to the database!"})}
                message_history.append(feedback_db)


                print("======================System Message============================")
                print("Sending back to main agent...")
                print("========================AI MESSAGE==============================")

                llm_response_complete = ""
                for chunk in client.chat.completions.create(model=model, max_tokens=max_tokens, temperature=temperature, stream=True, messages=message_history):

                    response = chunk.choices[0].delta.content or ""

                    print(response, end="", flush=True)
                    llm_response_complete += response

                print()

            elif db_agent_resp.get('task').lower().strip() == "retrieve":

                title_note = db_agent_resp.get('title').strip()

                if title_note.lower() == "none":
                    all_collections = [c.get("title") for c in collections.find({}, {"_id": 0, "title": 1})]

                    feedback_db = {"role": "system", "content": str({"sender": "Database Agent", "retreived_all_title_saved_notes": all_collections})}
                    message_history.append(feedback_db)

                    print("======================System Message============================")
                    print("Sending back to main agent...")
                    print("========================AI MESSAGE==============================")

                    llm_response_complete = ""
                    for chunk in client.chat.completions.create(model=model, max_tokens=max_tokens, temperature=temperature, stream=True, messages=message_history):

                        response = chunk.choices[0].delta.content or ""

                        print(response, end="", flush=True)
                        llm_response_complete += response

                    print()

                else:
                    retrieved_docs = [c for c in collections.find({"title": title_note}, {"_id": 0})]

                    if len(retrieved_docs) == 1:
                        title_note = retrieved_docs[-1]["title"]
                        content_note = retrieved_docs[-1]["content"]

                        feedback_db =  {"role": "system", "content": str({"sender": "Database Agent", "retreived_titles": title_note, "retrieved_content": content_note})}
                        message_history.append(feedback_db)

                        print("======================System Message============================")
                        print("Sending back to main agent...")
                        print("========================AI MESSAGE==============================")

                        llm_response_complete = ""
                        for chunk in client.chat.completions.create(model=model, max_tokens=max_tokens, temperature=temperature, stream=True, messages=message_history):

                            response = chunk.choices[0].delta.content or ""

                            print(response, end="", flush=True)
                            llm_response_complete += response

                        print()

            elif db_agent_resp.get('task').lower().strip() == "delete":
                title_note = db_agent_resp.get('title').strip()
                status_deletion = collections.find_one_and_delete({"title": title_note}, {"_id": 0})

                if status_deletion:

                    feedback_db =  {"role": "system", "content": str({"sender": "Database Agent", "deleted_notes": status_deletion.get("title")})}
                    message_history.append(feedback_db)

                    print("======================System Message============================")
                    print("Sending back to main agent...")
                    print("========================AI MESSAGE==============================")

                    llm_response_complete = ""
                    for chunk in client.chat.completions.create(model=model, max_tokens=max_tokens, temperature=temperature, stream=True, messages=message_history):

                        response = chunk.choices[0].delta.content or ""

                        print(response, end="", flush=True)
                        llm_response_complete += response

                    print()

                else:

                    feedback_db =  {"role": "system", "content": str({"sender": "Database Agent", "deleted_notes": "No note exists in database (check again)"})}
                    message_history.append(feedback_db)

                    print("======================System Message============================")
                    print("Sending back to main agent...")
                    print("========================AI MESSAGE==============================")

                    llm_response_complete = ""
                    for chunk in client.chat.completions.create(model=model, max_tokens=max_tokens, temperature=temperature, stream=True, messages=message_history):

                        response = chunk.choices[0].delta.content or ""

                        print(response, end="", flush=True)
                        llm_response_complete += response

                    print()
                                    
    else:
        continue